# Generate Confusion Matrices

In [1]:
# Imports
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%config InlineBackend.figure_format = 'svg'

## Load the technology and its settings

Below, choose which technology to load data and settings for:

In [2]:
# Choose technology here
TECH = 'B'

# Load settings for technology
with open(f"data/tech{TECH}/settings.json") as sfile:
    settings = json.load(sfile)

In [3]:
# Load data for technology
colnames = ["addr", "time", "r", "g", "gi", "range", "timept"]
data = pd.read_csv(f"data/tech{TECH}/relaxdata.min.tsv.gz", names=colnames, sep='\t')
data.head()

,addr,time,r,g,gi,range,timept
0,0,0.0,394055.654398,0.000003,0.000003,0,0.0
1,2,0.0,197630.281970,0.000005,0.000005,1,0.0
2,4,0.0,118221.233040,0.000008,0.000008,2,0.0
3,6,0.0,61104.839289,0.000016,0.000016,4,0.0
4,8,0.0,59039.756070,0.000017,0.000017,4,0.0


In [12]:
# Specify the allocation
ranges = range(0, 32, 2) # Use every second range as the 4-bit allocation

# Clip to [0, gmax]
data["g"] = np.clip(data["g"], 0, settings["gmax"])
data["gi"] = np.clip(data["gi"], 0, settings["gmax"])

# Get the initial and final range at each timept
data["irange"] = np.int16((data["gi"] + settings["gmax"]/32)//(settings["gmax"]/16))
data["range"] = np.int16((data["g"] + settings["gmax"]/32)//(settings["gmax"]/16))

In [10]:
# Show some data
data.head(20)

,addr,time,r,g,gi,range,timept,irange
0,0,0.0,3.940557e+05,2.537713e-06,2.537713e-06,0,0.0,0
1,2,0.0,1.976303e+05,5.059953e-06,5.059953e-06,1,0.0,1
2,4,0.0,1.182212e+05,8.458717e-06,8.458717e-06,1,0.0,1
3,6,0.0,6.110484e+04,1.636532e-05,1.636532e-05,2,0.0,2
4,8,0.0,5.903976e+04,1.693774e-05,1.693774e-05,2,0.0,2
5,10,0.0,4.541739e+04,2.201800e-05,2.201800e-05,3,0.0,3
6,12,0.0,3.531740e+04,2.831465e-05,2.831465e-05,4,0.0,4
7,14,0.0,3.394491e+04,2.945950e-05,2.945950e-05,4,0.0,4
8,16,0.0,2.835015e+04,3.527318e-05,3.527318e-05,4,0.0,4
9,18,0.0,2.654618e+04,3.767020e-05,3.767020e-05,5,0.0,5


In [24]:
# Get confusion matrix
for timept in settings["times"]:
    confmat = data[data["timept"] == timept].groupby(["range", "irange"]).size().unstack(fill_value=0).to_numpy()
    print(timept)
    print(confmat)
    np.save(f"confmats/tech{TECH}/{timept}.npy", confmat)

array([[9193,  409,    5,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [ 785, 6834, 1623,  224,   53,    1,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [   9,  842, 5230, 1424,  503,  115,   27,    9,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [   0,   61,  912, 4368, 1741,  579,  202,   39,   11,    5,    0,
           0,    0,    0,    0,    0,    0],
       [   0,    9,  121,  936, 4177, 1439,  646,  173,   49,   19,    4,
           0,    0,    0,    0,    0,    0],
       [   0,    0,   21,  202,  890, 3425, 1616,  494,  144,   34,    4,
           0,    0,    0,    0,    0,    0],
       [   0,    0,    4,   60,  239,  853, 4441, 1125,  336,  106,    8,
           2,    0,    0,    0,    0,    0],
       [   0,    0,    0,   15,   52,  249,  869, 3429, 1344,  320,   34,
          14,    3,    0,    0,    0,    0],
       [   0,    0,    0,    0,    2,   61,  231